In [1]:
import pyspark
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import *
from pyspark.sql.functions import max
from pyspark.sql.functions import col
import pyspark.sql.functions as F
from pyspark.sql.types import StructType,StructField, StringType

# Creating the SparkSession
spark = SparkSession.builder.appName('Case_Study_Adidas').master("local").getOrCreate()

# 1. Load the data

In [2]:
# Load JSON file into dataframe

path = 'C:\\BigData\\ol_cdump.json'
data_df = spark.read.option("multiline", "false").option("inferSchema", "true").option("header", "true").json(path)

# Printing the Schema
# df.printSchema()

# 2. Make sure your data set is cleaned enough, so we for example don't include in results with empty/null "titles" and/or "number of pages" is greater than 20 and "publishing year" is after 1950. 

In [3]:
# Filtering the data

# df = data_df.filter(data_df.title.isNotNull()).filter(data_df.number_of_pages > '20').filter(data_df.publish_date > '1950')
df = data_df.filter((data_df.title.isNotNull()) & (data_df.number_of_pages > '20') & (data_df.publish_date > '1950'))

df.select("title","number_of_pages","publish_date").distinct().show(10,truncate=False)

+------------------------------------------------------------------------------------+---------------+----------------+
|title                                                                               |number_of_pages|publish_date    |
+------------------------------------------------------------------------------------+---------------+----------------+
|Hidden & Dangerous 2                                                                |240            |October 28, 2003|
|Facts of life and death                                                             |33             |1970            |
|The effect of two types of verbal hierarchy on problem solving                      |48             |1968            |
|American broadcasting                                                               |778            |1970            |
|Do you know your economic ABC's?                                                    |34             |1963            |
|The fundamentals of photoengraving     

# 3.1 Select all "Harry Potter" books

In [4]:
# Select all "Harry Potter" books

df.filter(col("title").contains("Harry Potter")).select("title","publish_date").show(truncate=False)

+----------------------------------------+-------------+
|title                                   |publish_date |
+----------------------------------------+-------------+
|Harry Potter and the philosopher's stone|1998         |
|Harry Potter y la piedra filosofal      |2000         |
|The Science of Harry Potter             |June 23, 2003|
+----------------------------------------+-------------+



# 3.2 Get the book with the most pages

In [5]:
# To Get the book with the most pages

df.createOrReplaceTempView("records")
spark.sql("select number_of_pages,title,publish_date from(select *, dense_rank() over(order by number_of_pages desc)r from records) where r=1").show(truncate=False)

+---------------+-----------------------------+------------+
|number_of_pages|title                        |publish_date|
+---------------+-----------------------------+------------+
|48418          |Nihon shokuminchi kenchikuron|2008        |
+---------------+-----------------------------+------------+



In [6]:
# To Get the book with the most pages(No of Pages)

df.groupby().max('number_of_pages').collect()[0].asDict()['max(number_of_pages)']

48418

# 3.3 Find the Top 5 authors with most written books (assuming author in first position in the array, "key" field and each row is a different book)

In [7]:
df.select("*").groupby("authors").max('number_of_pages').distinct().show(5,truncate=False)

+-----------------------------------+--------------------+
|authors                            |max(number_of_pages)|
+-----------------------------------+--------------------+
|[{null, /authors/OL4670954A, null}]|146                 |
|[{null, /authors/OL6536041A, null}]|650                 |
|[{null, /authors/OL6536108A, null}]|239                 |
|[{null, /authors/OL6286129A, null}]|160                 |
|[{null, /authors/OL1149330A, null}]|176                 |
+-----------------------------------+--------------------+
only showing top 5 rows



In [ ]:
sql_df = spark.sql("select authors from(select *, dense_rank() over(order by number_of_pages desc)r from records) where r between 1 and 5")

print(sql_df.show(5, truncate=False))

# 3.4 Find the Top 5 genres with most books

In [ ]:
# Find the Top 5 genres with most books

df_geners = df.filter(data_df.genres.isNotNull())

df_geners.select("genres").show(5, truncate=False)

In [ ]:
df_geners.createOrReplaceTempView("sample")

# spark.sql("select geners,count(*) from sample group by genres").show(5,truncate=False)

In [ ]:
df_geners.select("*").groupby("genres").sum().distinct().show(5,truncate=False)

# 3.5 Get the avg. number of pages

In [8]:
# Get the avg. number of pages
df.groupby().avg('number_of_pages').collect()[0].asDict()['avg(number_of_pages)']

231.90327568877092

In [9]:
df.groupby().avg('number_of_pages').show()

+--------------------+
|avg(number_of_pages)|
+--------------------+
|  231.90327568877092|
+--------------------+



In [ ]:
# Per publish year, get the number of authors that published at least one book

df.createOrReplaceTempView("records")
spark.sql("select to_date(publish_date) as date from records").show(10)